In [1]:
from solid.utils import *
from solid.objects import *
from solid.solidpython import *
import viewscad
import subprocess
import traceback

r = viewscad.Renderer(openscad_exec='C:\Program Files\OpenSCAD\openscad')


In [2]:
ARM_STL = 'D:/Projects/HandEyeCoordination/models/V3/hand_v2.stl'
ARM_BOUNDS = [130, 150, 500]

In [3]:
move = translate

In [4]:
def cutInHalf(model, size, half):
    [x,y,z] = size
    bounds = translate([-x/2, -y/2, 0])(cube(size))
    whole = intersection()(model, bounds)
    top_size = [x,y,z*(1-half)]
    top_bounds = translate([-x/2, -y/2, z*half])(cube(top_size))
    top = translate([0,0,-z*half])(intersection()(whole, top_bounds))
    bottom_size = [x,y,z*half]
    bottom_bounds = translate([-x/2, -y/2, 0])(cube(bottom_size))
    bottom = intersection()(whole, bottom_bounds)
    return (top, top_size), (bottom, bottom_size)

In [5]:
bearing_ir = 4
bearing_or = 22.4/2
bearing_h = 7
bearing = cylinder(r=bearing_or, h=bearing_h) + hole()(cylinder(r=bearing_ir, h=bearing_h))


In [6]:
servo_gear_r = 2.5
servo_gear_h = 3
servo_x = 12.3
servo_y = 22.8
servo_z = 22.8
servo_arm_w = 32.3
servo_arm_l = (servo_arm_w - servo_y)/2
servo_arm_h = 11

servo_arm_2d = union()(
    circle(d=7),
    translate([0,14])(circle(d=4)),
    polygon([[3,0],[2,14],[-2,14],[-3,0]])
)

servo_arm = linear_extrude(height=5)(
    servo_arm_2d
)

detailed_servo_arm_neg = move([0,0,-5])(
    union()(
        move([0,0,-1])(linear_extrude(height=3)(servo_arm_2d)),
        cylinder(d=7, h=6)
    )
)

servo_wing_neg = union()(
    cube([servo_x, servo_arm_l, servo_z-2]),
    move([servo_x/2, 2, 0])(cylinder(r=1, h=servo_z+10))
)

servo_neg = move([-servo_x/2, -servo_x/2, -servo_z-6])(
    union()(
        cube([servo_x, servo_y, servo_z]),
        scale([1,-1,1])(move([0,0,0])(servo_wing_neg)),
        move([0,servo_y,0])(servo_wing_neg),
        move([servo_x/2,servo_x/2,0])(
            union()(
                cylinder(r=servo_x/2, h=servo_z+6),
                move([0,servo_x/2+1,0])(
                    cylinder(r=2.5, h=servo_z+6)
                )
            )
        )
    )
)

r.render(detailed_servo_arm_neg)

In [7]:
arm = translate([-40,18,0])(import_stl(ARM_STL))
(hand_cut, hand_size), (forearm_cut, forearm_size) = cutInHalf(arm, ARM_BOUNDS, 0.5)

r.render(hand_cut)

In [8]:
forearm = forearm_cut
forearm = difference()(forearm, scale(0.9)(forearm))
forearm -= translate([0,0,forearm_size[2]-bearing_h*4])(
    scale([1,1,4])(
        bearing
    )
)
forearm -= translate([0,0,forearm_size[2]-bearing_h-10])(
    cylinder(r=15, h=10)
)
servo_hole = (
    translate([-servo_x/2,-servo_x/2,0])(
        cube([servo_x, servo_y, servo_z+7])
    ) 
    +
    translate([-servo_x/2, -servo_x/2-servo_arm_l, 0])(
        cube([servo_x, servo_arm_w, servo_z-1])
    )
)
forearm -= translate([0,0,forearm_size[2]-46])(servo_hole)
(forearm_top, forearm_top_size), (forearm_bottom, forearm_bottom_size) = cutInHalf(forearm, forearm_size, 0.5)
scad_render_to_file(forearm_top, 'D:/Projects/HandEyeCoordination/models/V3/forearm_top.scad')
scad_render_to_file(forearm_bottom, 'D:/Projects/HandEyeCoordination/models/V3/forearm_bottom.scad')
print(forearm_bottom_size)
r.render(forearm_bottom)

[130, 150, 125.0]


In [9]:
axel = cylinder(r=10, h=8)
axel += cylinder(r=bearing_ir+2, h=10)
axel += cylinder(r=bearing_ir, h=60)
axel -= servo_arm
# axel -= move([0,0,25])(rotate([0,90,0])(cylinder(r=1,h=20, center=True)))

scad_render_to_file(axel, 'D:/Projects/HandEyeCoordination/models/V3/axel.scad')


r.render(axel)

In [10]:
hand = hand_cut
segments = 100
camera_r = 9

eye_gimbal_key = cylinder(r=bearing_ir, h=20)
eye_gimbal_key += move([-4,-8,0])(cube([8,16,3]))

eye_gimbal_keyhole = cylinder(r=bearing_ir, h=10)
eye_gimbal_keyhole += move([-4.3,-8.3,0])(cube([8.6,16.6,6]))



eye_greater = sphere(r=50, segments=segments)
eye_gimbal = (eye_greater - sphere(r=46, segments=segments)) * move([-50,-50,-10])(cube([100,100,30]))
eye_gimbal -= move([0,45,0])(rotate([0,-90,90])(detailed_servo_arm_neg))
eye_gimbal -= move([0,-50,0])(rotate([0,90,90])(eye_gimbal_keyhole))
# eye_gimbal += move([0,-50,0])(rotate([0,90,90])(eye_gimbal_key))

eye_gimbal = rotate([0,0,180])(eye_gimbal) # wrong direction on the eye

eye = sphere(r=45, segments=segments)
eye -= sphere(r=41, segments=segments)
eye *= move([-50,-50,-20])(cube([100,100,100]))
eye -= cylinder(r=camera_r, h=100)

camera_peg = cylinder(r=3, h=10) - (cylinder(r=1,h=10))
camera_pegs = move([14,14,0])(camera_peg) + move([14,-14,0])(camera_peg) + move([-14,14,0])(camera_peg) + move([-14,-14,0])(camera_peg)
camera_pegs += cylinder(r=40, h=20) - move([-15,-15,0])(cube([30,30,20]))

eye += translate([0,0,34])(camera_pegs)

servo_block_h = 6
servo_block_w = 10
servo_block = cube([servo_x, servo_block_w, servo_block_h]) + hole()(move([servo_x/2,2,0])(cylinder(r=1, h=servo_block_h)))
servo_holder = move([0,servo_x/2,0])(
    union()(
        move([-servo_x/2,-servo_y,0])(
            scale([1,-1,1])( # mirror it so it faces the other direction
                servo_block
            )
        ),
        move([-servo_x/2,0,0])(
            servo_block
        )
    )
)

servo_hole = cylinder(r=servo_x/2, h=40) + move([0,-servo_x/2,0])(cylinder(r=3, h=40))

eye -= translate([45,0,0])(rotate([0,90,0])(servo_neg))
# eye -= translate([30,0,0])(rotate([0,90,0])(servo_hole))
eye -= translate([-43,0,0])(rotate([0,90,0])(bearing))
eye -= translate([-42,0,0])(rotate([0,90,0])(cylinder(r=bearing_ir+1, h=20, center=True)))

eye *= sphere(r=45, segments=segments) # remove stuff sticking out the sides
eye = rotate([0,0,90])(eye) # I wrote this in the wrong direction and need to turn it to use it

scad_render_to_file(eye, 'D:/Projects/HandEyeCoordination/models/V3/eye.scad')
scad_render_to_file(eye_gimbal, 'D:/Projects/HandEyeCoordination/models/V3/eye_gimbal.scad')
scad_render_to_file(eye_gimbal_key, 'D:/Projects/HandEyeCoordination/models/V3/eye_gimbal_key.scad')

just_the_eye = eye_gimbal + rotate([180,0,0])(eye)

hand = hand - translate([25,7,173])(eye_greater)
hand_and_eye = hand
hand_and_eye += translate([25,7,173])(
    rotate([0,-110,-5])(
        just_the_eye
    )
)

r.render(eye)

In [11]:
keyhole_h = 20-bearing_h
# hand = hand_cut
wrist_keyhole = cylinder(r=bearing_ir+0.5, h=keyhole_h)
wrist_keyhole += move([0,0,keyhole_h/2])(cube([4,30,keyhole_h], center=True))
wrist_keyhole += move([0,0,keyhole_h/2])(cube([30,4,keyhole_h], center=True))

hand -= wrist_keyhole

hand -= move([0,0,130])(rotate([0,-90,0])(cylinder(r=8,h=60)))

(hand_top, hand_top_size), (hand_bottom, hand_bottom_size) = cutInHalf(hand, hand_size, 0.5)
scad_render_to_file(hand_top, 'D:/Projects/HandEyeCoordination/models/V3/hand_top.scad')
scad_render_to_file(hand_bottom, 'D:/Projects/HandEyeCoordination/models/V3/hand_bottom.scad')

print(hand_top_size)
r.render(hand_top)

[130, 150, 125.0]
